# 断点回归 (RDD) - 门槛处的自然实验

**学习目标**：
1. 理解 RDD 的核心思想：利用「门槛」创造的局部随机化
2. 掌握 Sharp RDD 和 Fuzzy RDD 的区别与应用
3. 学习带宽选择、稳健推断和有效性检验方法

---

## 引子：满 200 减 50 的秘密

想象你是电商平台的数据分析师，运营团队推出了「满 200 减 50」的优惠券。你想知道：**这 50 元优惠真的提升了用户的复购率吗？**

### 挑战
- ❌ 不能直接对比「用券用户」vs「未用券用户」—— 他们本身就不同（选择偏差）
- ❌ 不能做随机实验 —— 优惠券已经发放，无法撤回

### 洞察
但你注意到一个有趣的现象：
- 消费 **199 元**的用户：没券
- 消费 **201 元**的用户：有券

**这两个用户几乎一样！** 只是因为 2 元的差距，一个刚好没达标，一个刚好达标。在门槛处（200 元），用券与否近似**随机**。

这就是 **断点回归 (Regression Discontinuity Design, RDD)** 的核心思想。

---

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from scipy.optimize import minimize_scalar
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 配置
COLORS = {
    'primary': '#2D9CDB',
    'secondary': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'light': '#E0E0E0'
}

print("✅ 环境准备完成")

---

## Part 1: RDD 的直觉 —— 门槛创造的局部随机化

### 1.1 核心思想

RDD 利用政策、规则产生的 **不连续性（门槛）** 来识别因果效应。

**关键假设**：在门槛附近，个体的特征是**连续的**，只有处理状态发生跳跃。

### 比喻：考试及格线

- 考试分数 59 分 vs 60 分，能力差别微乎其微
- 但 60 分及格，可以获得奖学金；59 分不及格，没有奖学金
- 在 60 分这个门槛处，「是否获得奖学金」近似随机分配
- 对比 59 分和 60 分学生的后续表现，可以估计奖学金的因果效应

### 1.2 形式化定义

**Running Variable（驱动变量）** $X$：决定是否接受处理的连续变量（如消费金额、考试分数）

**Cutoff（门槛）** $c$：处理分配的临界点（如 200 元、60 分）

**Treatment（处理）** $D$：
$$
D_i = \begin{cases}
1 & \text{if } X_i \geq c \\
0 & \text{if } X_i < c
\end{cases}
$$

**因果效应（在门槛处）**：
$$
\tau_{RDD} = \lim_{x \downarrow c} E[Y_i | X_i = x] - \lim_{x \uparrow c} E[Y_i | X_i = x]
$$

即：门槛右侧的期望结果 - 门槛左侧的期望结果。

### 1.3 为什么断点处可比？

**连续性假设**：如果没有处理，结果变量在门槛处应该是连续的。

$$
\lim_{x \downarrow c} E[Y_i(0) | X_i = x] = \lim_{x \uparrow c} E[Y_i(0) | X_i = x]
$$

这意味着：
- 在门槛附近，除了处理状态，其他特征都是连续的
- 门槛两侧的个体**本质上相同**（局部随机化）
- 结果的跳跃完全归因于处理

### 1.4 RDD vs 随机实验

| 维度 | 随机实验 | RDD |
|------|----------|-----|
| 可比性范围 | 全局 | 局部（门槛附近） |
| 外部效度 | 高 | 低（只能推断门槛处） |
| 内部效度 | 高 | 中（依赖假设） |
| 实施成本 | 高 | 低（观察性） |
| 适用场景 | 可控环境 | 已有政策/规则 |

---

In [ ]:
# 可视化：RDD 的核心直觉

def simulate_rdd_data(n=500, cutoff=200, tau=15, noise=20):
    """
    模拟 RDD 数据
    
    参数:
        n: 样本量
        cutoff: 门槛值
        tau: 处理效应
        noise: 噪声标准差
    """
    # 驱动变量 (消费金额)
    X = np.random.uniform(100, 300, n)
    
    # 处理状态 (是否获得优惠券)
    D = (X >= cutoff).astype(int)
    
    # 潜在结果 (复购率, %)
    # Y(0): 基线 + X 的线性效应
    Y0 = 30 + 0.1 * (X - cutoff) + np.random.normal(0, noise, n)
    
    # Y(1): 在 Y(0) 基础上加处理效应
    Y1 = Y0 + tau
    
    # 观测结果
    Y = D * Y1 + (1 - D) * Y0
    
    return pd.DataFrame({
        'spending': X,
        'coupon': D,
        'repurchase_rate': Y,
        'Y0': Y0,
        'Y1': Y1
    })

# 生成数据
df = simulate_rdd_data(n=1000, cutoff=200, tau=15)

# 可视化
fig = go.Figure()

# 散点图
fig.add_trace(go.Scatter(
    x=df[df['coupon']==0]['spending'],
    y=df[df['coupon']==0]['repurchase_rate'],
    mode='markers',
    name='未获券 (X < 200)',
    marker=dict(color=COLORS['danger'], size=5, opacity=0.5)
))

fig.add_trace(go.Scatter(
    x=df[df['coupon']==1]['spending'],
    y=df[df['coupon']==1]['repurchase_rate'],
    mode='markers',
    name='获券 (X ≥ 200)',
    marker=dict(color=COLORS['secondary'], size=5, opacity=0.5)
))

# 拟合线 (分段线性)
left = df[df['spending'] < 200]
right = df[df['spending'] >= 200]

# 左侧拟合
X_left = left['spending'].values.reshape(-1, 1)
y_left = left['repurchase_rate'].values
model_left = LinearRegression().fit(X_left, y_left)
x_left_line = np.linspace(100, 200, 100).reshape(-1, 1)
y_left_pred = model_left.predict(x_left_line)

fig.add_trace(go.Scatter(
    x=x_left_line.flatten(),
    y=y_left_pred,
    mode='lines',
    name='左侧拟合',
    line=dict(color=COLORS['danger'], width=3)
))

# 右侧拟合
X_right = right['spending'].values.reshape(-1, 1)
y_right = right['repurchase_rate'].values
model_right = LinearRegression().fit(X_right, y_right)
x_right_line = np.linspace(200, 300, 100).reshape(-1, 1)
y_right_pred = model_right.predict(x_right_line)

fig.add_trace(go.Scatter(
    x=x_right_line.flatten(),
    y=y_right_pred,
    mode='lines',
    name='右侧拟合',
    line=dict(color=COLORS['secondary'], width=3)
))

# 门槛线
fig.add_vline(x=200, line_dash="dash", line_color="black", 
              annotation_text="门槛: 200 元")

# 标注跳跃
y_left_at_cutoff = model_left.predict([[200]])[0]
y_right_at_cutoff = model_right.predict([[200]])[0]
tau_est = y_right_at_cutoff - y_left_at_cutoff

fig.add_annotation(
    x=200, y=(y_left_at_cutoff + y_right_at_cutoff) / 2,
    text=f"跳跃 ≈ {tau_est:.1f}%",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor=COLORS['primary']
)

fig.update_layout(
    title='RDD 核心直觉：门槛处的跳跃就是因果效应',
    xaxis_title='消费金额 (元)',
    yaxis_title='复购率 (%)',
    template='plotly_white',
    hovermode='closest',
    height=500
)

fig.show()

print(f"\n📊 估计的处理效应: {tau_est:.2f}%")
print(f"   真实处理效应: 15.00%")

---

## Part 2: Sharp RDD —— 门槛完全决定处理

### 2.1 定义

**Sharp RDD**：处理状态完全由驱动变量和门槛决定，没有例外。

$$
P(D_i = 1 | X_i = x) = \begin{cases}
1 & \text{if } x \geq c \\
0 & \text{if } x < c
\end{cases}
$$

**例子**：
- ✅ 法定饮酒年龄 21 岁
- ✅ 考试 60 分及格
- ✅ 满 200 减 50（系统自动）

### 2.2 估计方法：局部多项式回归

**基本思路**：在门槛附近用多项式拟合 $E[Y|X]$，估计跳跃大小。

#### 线性规范（最常用）

$$
Y_i = \alpha + \tau D_i + \beta_1 (X_i - c) + \beta_2 D_i \cdot (X_i - c) + \epsilon_i
$$

其中：
- $\tau$: 处理效应（门槛处的跳跃）
- $\beta_1$: 左侧斜率
- $\beta_1 + \beta_2$: 右侧斜率
- $(X_i - c)$: 中心化，使 $\tau$ 代表门槛处的效应

#### 高阶多项式

$$
Y_i = \alpha + \tau D_i + \sum_{p=1}^{P} \left[ \beta_p (X_i - c)^p + \gamma_p D_i \cdot (X_i - c)^p \right] + \epsilon_i
$$

**注意**：高阶多项式可能过拟合，通常 $P \leq 2$。

### 2.3 带宽选择

**带宽 $h$**：只使用 $|X_i - c| \leq h$ 的样本进行估计。

**权衡**：
- 小带宽 → 低偏差（更接近门槛），高方差（样本少）
- 大带宽 → 高偏差（远离门槛），低方差（样本多）

**最优带宽**：最小化均方误差 (MSE)

$$
h^* = \arg\min_h \text{MSE}(\hat{\tau}) = \arg\min_h \left[ \text{Bias}(\hat{\tau})^2 + \text{Var}(\hat{\tau}) \right]
$$

**实践中**：使用 Imbens-Kalyanaraman (2012) 或 Calonico-Cattaneo-Titiunik (2014) 方法自动选择。

---

In [ ]:
# Sharp RDD 估计器

class SharpRDD:
    """
    Sharp RDD 估计器（局部多项式回归）
    """
    def __init__(self, cutoff=0, bandwidth=None, polynomial_order=1, kernel='triangular'):
        self.cutoff = cutoff
        self.bandwidth = bandwidth
        self.polynomial_order = polynomial_order
        self.kernel = kernel
        self.tau_ = None
        self.se_ = None
        
    def _kernel_weight(self, x):
        """核权重函数"""
        u = x / self.bandwidth
        if self.kernel == 'uniform':
            return (np.abs(u) <= 1).astype(float)
        elif self.kernel == 'triangular':
            return np.maximum(0, 1 - np.abs(u))
        elif self.kernel == 'epanechnikov':
            return np.maximum(0, 0.75 * (1 - u**2))
        else:
            raise ValueError(f"Unknown kernel: {self.kernel}")
    
    def _select_bandwidth_ik(self, X, Y, D):
        """
        Imbens-Kalyanaraman (2012) 带宽选择
        简化版实现
        """
        # 使用全样本估计
        X_centered = X - self.cutoff
        
        # 分别拟合左右两侧
        left_mask = D == 0
        right_mask = D == 1
        
        # 估计方差
        var_left = np.var(Y[left_mask])
        var_right = np.var(Y[right_mask])
        
        # 估计二阶导数（用三阶多项式）
        poly = PolynomialFeatures(degree=3)
        X_poly_left = poly.fit_transform(X_centered[left_mask].reshape(-1, 1))
        X_poly_right = poly.fit_transform(X_centered[right_mask].reshape(-1, 1))
        
        model_left = LinearRegression().fit(X_poly_left, Y[left_mask])
        model_right = LinearRegression().fit(X_poly_right, Y[right_mask])
        
        # 简化公式（实际 IK 更复杂）
        n = len(X)
        range_x = np.max(X) - np.min(X)
        
        # 经验公式
        h_ik = 1.84 * np.sqrt(var_left + var_right) * n**(-1/5) * range_x
        
        return h_ik
    
    def fit(self, X, Y, D=None):
        """
        拟合 RDD 模型
        
        参数:
            X: 驱动变量 (n,)
            Y: 结果变量 (n,)
            D: 处理状态 (n,), 如果为 None 则根据 cutoff 自动生成
        """
        X = np.array(X)
        Y = np.array(Y)
        
        if D is None:
            D = (X >= self.cutoff).astype(int)
        else:
            D = np.array(D)
        
        # 自动选择带宽
        if self.bandwidth is None:
            self.bandwidth = self._select_bandwidth_ik(X, Y, D)
            print(f"自动选择带宽: h = {self.bandwidth:.2f}")
        
        # 筛选带宽内的样本
        mask = np.abs(X - self.cutoff) <= self.bandwidth
        X_bw = X[mask]
        Y_bw = Y[mask]
        D_bw = D[mask]
        
        # 核权重
        weights = self._kernel_weight(X_bw - self.cutoff)
        
        # 构建多项式特征
        X_centered = (X_bw - self.cutoff).reshape(-1, 1)
        
        # 特征矩阵: [1, D, X-c, D*(X-c), (X-c)^2, D*(X-c)^2, ...]
        features = [np.ones(len(X_bw)), D_bw]
        for p in range(1, self.polynomial_order + 1):
            features.append(X_centered.flatten() ** p)
            features.append(D_bw * (X_centered.flatten() ** p))
        
        X_design = np.column_stack(features)
        
        # 加权最小二乘
        W = np.diag(weights)
        beta = np.linalg.solve(X_design.T @ W @ X_design, X_design.T @ W @ Y_bw)
        
        # 提取处理效应（第二个系数）
        self.tau_ = beta[1]
        
        # 标准误（稳健标准误）
        residuals = Y_bw - X_design @ beta
        meat = X_design.T @ W @ np.diag(residuals**2) @ W @ X_design
        bread = np.linalg.inv(X_design.T @ W @ X_design)
        vcov = bread @ meat @ bread
        self.se_ = np.sqrt(vcov[1, 1])
        
        return self
    
    def summary(self, alpha=0.05):
        """输出估计结果"""
        if self.tau_ is None:
            raise ValueError("模型未拟合，请先调用 fit()")
        
        z = stats.norm.ppf(1 - alpha/2)
        ci_lower = self.tau_ - z * self.se_
        ci_upper = self.tau_ + z * self.se_
        t_stat = self.tau_ / self.se_
        p_value = 2 * (1 - stats.norm.cdf(np.abs(t_stat)))
        
        print("\n" + "="*60)
        print("Sharp RDD 估计结果")
        print("="*60)
        print(f"带宽 (h):           {self.bandwidth:.2f}")
        print(f"多项式阶数:          {self.polynomial_order}")
        print(f"核函数:             {self.kernel}")
        print("-"*60)
        print(f"处理效应 (τ):       {self.tau_:.4f}")
        print(f"标准误 (SE):        {self.se_:.4f}")
        print(f"t 统计量:           {t_stat:.4f}")
        print(f"p 值:              {p_value:.4f}")
        print(f"{int((1-alpha)*100)}% 置信区间:      [{ci_lower:.4f}, {ci_upper:.4f}]")
        print("="*60)
        
        return {
            'tau': self.tau_,
            'se': self.se_,
            'ci': (ci_lower, ci_upper),
            'p_value': p_value
        }

# 测试 Sharp RDD 估计器
rdd = SharpRDD(cutoff=200, polynomial_order=1)
rdd.fit(df['spending'], df['repurchase_rate'])
results = rdd.summary()

print(f"\n✅ 真实效应: 15.00%")
print(f"✅ 估计偏差: {results['tau'] - 15:.2f}%")

### 🎯 TODO 1: 带宽敏感性分析

不同带宽会影响估计结果。请完成下面的代码，绘制不同带宽下的估计效应。

In [ ]:
# ===================================# TODO 1: 带宽敏感性分析 - 完整实现# ===================================bandwidths = np.linspace(10, 100, 20)tau_estimates = []ci_lower_list = []ci_upper_list = []se_list = []for h in bandwidths:    # 用不同带宽拟合 RDD 模型    rdd_temp = SharpRDD(cutoff=200, bandwidth=h, polynomial_order=1)    rdd_temp.fit(df['score'].values, df['conversion'].values)        tau_estimates.append(rdd_temp.tau_)    se_list.append(rdd_temp.se_)    ci_lower_list.append(rdd_temp.tau_ - 1.96 * rdd_temp.se_)    ci_upper_list.append(rdd_temp.tau_ + 1.96 * rdd_temp.se_)tau_estimates = np.array(tau_estimates)se_list = np.array(se_list)ci_lower_list = np.array(ci_lower_list)ci_upper_list = np.array(ci_upper_list)# 可视化fig = make_subplots(    rows=1, cols=2,    subplot_titles=('处理效应估计', '标准误'))# 左图：点估计 + 置信区间fig.add_trace(    go.Scatter(        x=bandwidths, y=tau_estimates,        mode='lines+markers',        name='RDD 估计',        line=dict(color=COLORS['treated'], width=2),        marker=dict(size=6),    ),    row=1, col=1)# 置信区间fig.add_trace(    go.Scatter(        x=np.concatenate([bandwidths, bandwidths[::-1]]),        y=np.concatenate([ci_upper_list, ci_lower_list[::-1]]),        fill='toself',        fillcolor='rgba(235, 87, 87, 0.2)',        line=dict(color='rgba(255,255,255,0)'),        showlegend=False,        name='95% CI',    ),    row=1, col=1)# 零线fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)# 右图：标准误fig.add_trace(    go.Scatter(        x=bandwidths, y=se_list,        mode='lines+markers',        name='标准误',        line=dict(color=COLORS['synthetic'], width=2),        marker=dict(size=6),        showlegend=False    ),    row=1, col=2)fig.update_xaxes(title_text="带宽 (h)", row=1, col=1)fig.update_xaxes(title_text="带宽 (h)", row=1, col=2)fig.update_yaxes(title_text="处理效应", row=1, col=1)fig.update_yaxes(title_text="标准误", row=1, col=2)fig.update_layout(    title_text='带宽敏感性分析',    template='plotly_white',    height=400,    showlegend=True)fig.show()# 打印结果表格print("=" * 70)print("带宽敏感性分析结果")print("=" * 70)print(f"{'带宽':>8s} {'效应':>10s} {'标准误':>10s} {'95% CI':>25s} {'显著':>6s}")print("-" * 70)for i in range(0, len(bandwidths), 4):  # 每隔4个显示    h = bandwidths[i]    tau = tau_estimates[i]    se = se_list[i]    ci_l = ci_lower_list[i]    ci_u = ci_upper_list[i]    sig = '***' if abs(tau/se) > 2.58 else ('**' if abs(tau/se) > 1.96 else '*' if abs(tau/se) > 1.65 else '')        print(f"{h:8.1f} {tau:10.4f} {se:10.4f} [{ci_l:8.4f}, {ci_u:8.4f}] {sig:>6s}")print("=" * 70)print("\n💡 解读:")print(f"  • 估计范围: [{min(tau_estimates):.4f}, {max(tau_estimates):.4f}]")print(f"  • 标准误范围: [{min(se_list):.4f}, {max(se_list):.4f}]")print(f"  • 带宽越小 → 偏差小但方差大（样本少）")print(f"  • 带宽越大 → 方差小但偏差大（包含远离断点的观测）")# 检查稳健性cv = np.std(tau_estimates) / abs(np.mean(tau_estimates))print(f"  • 估计的变异系数: {cv:.2f}")if cv < 0.2:    print(f"    ✓ 结果对带宽不敏感，估计较为稳健")else:    print(f"    ⚠ 结果对带宽较敏感，需谨慎解释")

---

## Part 3: Fuzzy RDD —— 门槛影响处理概率

### 3.1 从 Sharp 到 Fuzzy

现实中，门槛并不总是完全决定处理：
- 有些人消费超过 200 元，但**忘记使用优惠券**
- 有些人消费不足 200 元，但通过**客服申诉**获得了优惠券

这就是 **Fuzzy RDD**。

### 3.2 定义

**Fuzzy RDD**：门槛处处理概率发生跳跃，但不是 0 → 1 的完全跳跃。

$$
\lim_{x \downarrow c} P(D_i = 1 | X_i = x) \neq \lim_{x \uparrow c} P(D_i = 1 | X_i = x)
$$

但跳跃幅度 < 1：
$$
0 < \lim_{x \downarrow c} P(D_i = 1 | X_i = x) - \lim_{x \uparrow c} P(D_i = 1 | X_i = x) < 1
$$

**例子**：
- 奖学金政策：60 分及格**有资格**申请，但不是所有人都申请
- 优惠券：满 200 元**可用**，但有人忘记用

### 3.3 Fuzzy RDD 与 IV 的关系

Fuzzy RDD 本质上是一个 **工具变量 (IV)** 设计：

- **工具变量 $Z_i$**：是否超过门槛（$Z_i = \mathbb{1}[X_i \geq c]$）
- **处理变量 $D_i$**：是否实际接受处理
- **结果变量 $Y_i$**：感兴趣的结果

IV 假设：
1. **相关性**：$Z$ 影响 $D$（门槛处处理概率跳跃）
2. **排他性**：$Z$ 只通过 $D$ 影响 $Y$（门槛本身不直接影响结果）
3. **单调性**：$Z=1$ 不会让某些人反而不接受处理

### 3.4 估计方法：Two-Stage Least Squares (2SLS)

**第一阶段**：用 $Z$ 预测 $D$
$$
D_i = \pi_0 + \pi_1 Z_i + \pi_2 (X_i - c) + \pi_3 Z_i \cdot (X_i - c) + \nu_i
$$

**第二阶段**：用 $\hat{D}$ 估计效应
$$
Y_i = \alpha + \tau \hat{D}_i + \beta_1 (X_i - c) + \beta_2 Z_i \cdot (X_i - c) + \epsilon_i
$$

**Wald 估计量**（等价）：
$$
\tau_{Fuzzy} = \frac{\lim_{x \downarrow c} E[Y_i | X_i = x] - \lim_{x \uparrow c} E[Y_i | X_i = x]}{\lim_{x \downarrow c} E[D_i | X_i = x] - \lim_{x \uparrow c} E[D_i | X_i = x]} = \frac{\text{结果的跳跃}}{\text{处理的跳跃}}
$$

### 3.5 LATE 解释

Fuzzy RDD 估计的是 **Local Average Treatment Effect (LATE)**，即「顺从者」的效应：

- **顺从者 (Compliers)**：超过门槛就接受处理，低于门槛就不接受（$D_i(1) = 1, D_i(0) = 0$）
- **总是接受者 (Always-takers)**：无论是否超过门槛都接受处理（$D_i(1) = D_i(0) = 1$）
- **从不接受者 (Never-takers)**：无论是否超过门槛都不接受（$D_i(1) = D_i(0) = 0$）

$$
\tau_{LATE} = E[Y_i(1) - Y_i(0) | \text{Complier}]
$$

---

In [ ]:
# Fuzzy RDD 模拟与估计

def simulate_fuzzy_rdd(n=1000, cutoff=200, tau=15, compliance_rate=0.7):
    """
    模拟 Fuzzy RDD 数据
    
    参数:
        compliance_rate: 顺从率（超过门槛后实际使用优惠券的比例）
    """
    # 驱动变量
    X = np.random.uniform(100, 300, n)
    
    # 工具变量（是否超过门槛）
    Z = (X >= cutoff).astype(int)
    
    # 处理状态（不完全顺从）
    # 超过门槛: compliance_rate 的概率使用券
    # 未超过门槛: 0.1 的概率也能用券（申诉）
    D = np.zeros(n)
    D[Z == 1] = np.random.binomial(1, compliance_rate, (Z == 1).sum())
    D[Z == 0] = np.random.binomial(1, 0.1, (Z == 0).sum())
    
    # 结果变量
    Y0 = 30 + 0.1 * (X - cutoff) + np.random.normal(0, 20, n)
    Y1 = Y0 + tau
    Y = D * Y1 + (1 - D) * Y0
    
    return pd.DataFrame({
        'spending': X,
        'eligible': Z,  # 是否有资格
        'coupon_used': D,  # 是否实际使用
        'repurchase_rate': Y
    })

# 生成 Fuzzy RDD 数据
df_fuzzy = simulate_fuzzy_rdd(n=1000, cutoff=200, tau=15, compliance_rate=0.7)

# 可视化：处理状态的跳跃
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['(a) 处理概率的跳跃（第一阶段）', '(b) 结果的跳跃（约化式）']
)

# 左图：处理概率
bins_x = np.linspace(100, 300, 30)
bin_centers = (bins_x[:-1] + bins_x[1:]) / 2
treatment_prob = []

for i in range(len(bins_x) - 1):
    mask = (df_fuzzy['spending'] >= bins_x[i]) & (df_fuzzy['spending'] < bins_x[i+1])
    if mask.sum() > 0:
        treatment_prob.append(df_fuzzy[mask]['coupon_used'].mean())
    else:
        treatment_prob.append(np.nan)

fig.add_trace(
    go.Scatter(
        x=bin_centers,
        y=treatment_prob,
        mode='markers',
        marker=dict(size=8, color=COLORS['primary']),
        name='处理概率'
    ),
    row=1, col=1
)

fig.add_vline(x=200, line_dash="dash", line_color="black", row=1, col=1)

# 右图：结果的跳跃
avg_outcome = []
for i in range(len(bins_x) - 1):
    mask = (df_fuzzy['spending'] >= bins_x[i]) & (df_fuzzy['spending'] < bins_x[i+1])
    if mask.sum() > 0:
        avg_outcome.append(df_fuzzy[mask]['repurchase_rate'].mean())
    else:
        avg_outcome.append(np.nan)

fig.add_trace(
    go.Scatter(
        x=bin_centers,
        y=avg_outcome,
        mode='markers',
        marker=dict(size=8, color=COLORS['secondary']),
        name='平均结果'
    ),
    row=1, col=2
)

fig.add_vline(x=200, line_dash="dash", line_color="black", row=1, col=2)

fig.update_xaxes(title_text="消费金额 (元)", row=1, col=1)
fig.update_xaxes(title_text="消费金额 (元)", row=1, col=2)
fig.update_yaxes(title_text="使用优惠券概率", row=1, col=1)
fig.update_yaxes(title_text="复购率 (%)", row=1, col=2)

fig.update_layout(
    template='plotly_white',
    height=400,
    showlegend=False
)

fig.show()

# Fuzzy RDD 估计（2SLS）
from sklearn.linear_model import LinearRegression

# 只使用带宽内的样本
bandwidth = 50
mask_bw = np.abs(df_fuzzy['spending'] - 200) <= bandwidth
df_bw = df_fuzzy[mask_bw].copy()

# 构建特征
df_bw['X_centered'] = df_bw['spending'] - 200
df_bw['Z_X'] = df_bw['eligible'] * df_bw['X_centered']

# 第一阶段：Z -> D
X_first = df_bw[['eligible', 'X_centered', 'Z_X']].values
D_bw = df_bw['coupon_used'].values

first_stage = LinearRegression().fit(X_first, D_bw)
D_hat = first_stage.predict(X_first)

# 第二阶段：D_hat -> Y
X_second = np.column_stack([D_hat, df_bw[['X_centered', 'Z_X']].values])
Y_bw = df_bw['repurchase_rate'].values

second_stage = LinearRegression().fit(X_second, Y_bw)
tau_fuzzy = second_stage.coef_[0]

# Wald 估计（等价）
left_mask = (df_bw['spending'] < 200)
right_mask = (df_bw['spending'] >= 200)

# 结果的跳跃
reduced_form = df_bw[right_mask]['repurchase_rate'].mean() - df_bw[left_mask]['repurchase_rate'].mean()

# 处理的跳跃
first_stage_jump = df_bw[right_mask]['coupon_used'].mean() - df_bw[left_mask]['coupon_used'].mean()

tau_wald = reduced_form / first_stage_jump

print("\n" + "="*60)
print("Fuzzy RDD 估计结果")
print("="*60)
print(f"处理概率跳跃:       {first_stage_jump:.3f}")
print(f"结果跳跃:          {reduced_form:.3f}")
print("-"*60)
print(f"2SLS 估计:         {tau_fuzzy:.3f}%")
print(f"Wald 估计:         {tau_wald:.3f}%")
print(f"真实效应:          15.000%")
print("="*60)
print("\n💡 解释: Fuzzy RDD 估计的是顺从者 (Compliers) 的 LATE")

---

## Part 4: 带宽选择 —— 偏差-方差权衡

### 4.1 带宽的困境

**小带宽**：
- ✅ 优点：更接近门槛，线性近似更准确（低偏差）
- ❌ 缺点：样本少，估计不稳定（高方差）

**大带宽**：
- ✅ 优点：样本多，估计稳定（低方差）
- ❌ 缺点：远离门槛，线性近似不准（高偏差）

### 4.2 MSE-optimal 带宽

**目标**：最小化均方误差

$$
\text{MSE}(\hat{\tau}) = \text{Bias}(\hat{\tau})^2 + \text{Var}(\hat{\tau})
$$

**偏差**：与 $h^{p+1}$ 成正比（$p$ 是多项式阶数）

$$
\text{Bias}(\hat{\tau}) \approx C_1 \cdot h^{p+1}
$$

**方差**：与 $1/(n \cdot h)$ 成正比

$$
\text{Var}(\hat{\tau}) \approx \frac{C_2}{n \cdot h}
$$

**最优带宽**：

$$
h_{MSE}^* \propto n^{-1/(2p+3)}
$$

对于线性模型（$p=1$），$h^* \propto n^{-1/5}$。

### 4.3 实践方法

#### Imbens-Kalyanaraman (2012)
- 基于 MSE 最优化
- 适用于线性和二次多项式
- 需要估计条件方差和高阶导数

#### Calonico-Cattaneo-Titiunik (CCT, 2014)
- MSE-optimal 带宽
- 提供偏差校正的稳健推断
- 成为事实标准

### 4.4 稳健性检验

即使选择了"最优"带宽，仍需检验结果的稳健性：
- **带宽敏感性图**：绘制不同带宽下的估计值和置信区间
- **多种带宽**：报告 0.5h*, h*, 2h* 的结果
- **不同多项式**：线性、二次、三次的对比

---

### 🎯 TODO 2: 实现 CCT 带宽选择

Calonico-Cattaneo-Titiunik (2014) 提出了 MSE-optimal 带宽。请完成简化版的实现。

In [ ]:
# ===================================# TODO 2: CCT 带宽选择（简化版） - 完整实现# ===================================def cct_bandwidth(X, Y, cutoff, kernel='triangular'):    """    Calonico-Cattaneo-Titiunik (2014) MSE-optimal 带宽选择（简化实现）        参数:    ------    X : array-like        Running variable    Y : array-like        Outcome variable    cutoff : float        Cutoff point    kernel : str        Kernel type        返回:    ------    h_opt : float        Optimal bandwidth    """    X = np.asarray(X)    Y = np.asarray(Y)        # 中心化    X_centered = X - cutoff        # 分左右两侧    left_mask = X_centered < 0    right_mask = X_centered >= 0        X_left = X_centered[left_mask]    Y_left = Y[left_mask]    X_right = X_centered[right_mask]    Y_right = Y[right_mask]        # Step 1: 估计条件方差    # 用二次多项式回归的残差估计方差    from sklearn.linear_model import LinearRegression        # 左侧    if len(X_left) > 3:        poly_left = np.column_stack([X_left, X_left**2])        model_left = LinearRegression().fit(poly_left, Y_left)        resid_left = Y_left - model_left.predict(poly_left)        sigma2_left = np.var(resid_left)    else:        sigma2_left = np.var(Y_left)        # 右侧    if len(X_right) > 3:        poly_right = np.column_stack([X_right, X_right**2])        model_right = LinearRegression().fit(poly_right, Y_right)        resid_right = Y_right - model_right.predict(poly_right)        sigma2_right = np.var(resid_right)    else:        sigma2_right = np.var(Y_right)        # Step 2: 估计二阶导数    # 用三次多项式的二阶导数    if len(X_left) > 4:        poly3_left = np.column_stack([X_left, X_left**2, X_left**3])        model3_left = LinearRegression().fit(poly3_left, Y_left)        m2_left = 2 * model3_left.coef_[1]  # f''(0) = 2 * beta_2    else:        m2_left = 0.1        if len(X_right) > 4:        poly3_right = np.column_stack([X_right, X_right**2, X_right**3])        model3_right = LinearRegression().fit(poly3_right, Y_right)        m2_right = 2 * model3_right.coef_[1]    else:        m2_right = 0.1        # Step 3: 计算 MSE-optimal 带宽    # Fan & Gijbels (1996) 公式的简化版本    # h_opt = C * (sigma^2 / (n * m2^2))^(1/5)        n_left = len(X_left)    n_right = len(X_right)        # Kernel 常数    if kernel == 'triangular':        C_K = 3.44    else:        C_K = 2.70        # 计算左右两侧的最优带宽    if abs(m2_left) > 1e-6:        h_left = C_K * (sigma2_left / (n_left * m2_left**2))**(1/5)    else:        h_left = 30.0        if abs(m2_right) > 1e-6:        h_right = C_K * (sigma2_right / (n_right * m2_right**2))**(1/5)    else:        h_right = 30.0        # 取平均    h_opt = (h_left + h_right) / 2        # 限制在合理范围    h_opt = np.clip(h_opt, 10, 100)        return h_opt# 使用 CCT 带宽选择h_cct = cct_bandwidth(df['score'].values, df['conversion'].values, cutoff=200)print("=" * 70)print("CCT 带宽选择结果")print("=" * 70)print(f"CCT 最优带宽: h = {h_cct:.2f}")print("=" * 70)# 用 CCT 带宽重新估计rdd_cct = SharpRDD(cutoff=200, bandwidth=h_cct, polynomial_order=1)rdd_cct.fit(df['score'].values, df['conversion'].values)print(f"\n使用 CCT 带宽的 RDD 估计:")print(f"  处理效应: {rdd_cct.tau_:.4f}")print(f"  标准误: {rdd_cct.se_:.4f}")print(f"  t-统计量: {rdd_cct.tau_/rdd_cct.se_:.2f}")print(f"  p-value: {2*(1 - stats.norm.cdf(abs(rdd_cct.tau_/rdd_cct.se_))):.4f}")print(f"  95% CI: [{rdd_cct.tau_ - 1.96*rdd_cct.se_:.4f}, {rdd_cct.tau_ + 1.96*rdd_cct.se_:.4f}]")# 对比不同带宽print("\n" + "=" * 70)print("不同带宽下的估计对比")print("=" * 70)print(f"{'方法':<20s} {'带宽':>10s} {'效应':>10s} {'标准误':>10s}")print("-" * 70)for h, name in [(20, '小带宽'), (h_cct, 'CCT最优带宽'), (60, '大带宽')]:    rdd_temp = SharpRDD(cutoff=200, bandwidth=h, polynomial_order=1)    rdd_temp.fit(df['score'].values, df['conversion'].values)    print(f"{name:<20s} {h:>10.1f} {rdd_temp.tau_:>10.4f} {rdd_temp.se_:>10.4f}")print("=" * 70)print("\n💡 解读:")print("  • CCT 带宽在理论上最小化 MSE（均方误差）")print("  • 它在偏差和方差之间取得最优平衡")print("  • 实践中，建议报告多个带宽的结果，检查稳健性")

---

## Part 5: 检验方法 —— 验证 RDD 假设

RDD 的可信度依赖于关键假设。我们需要一系列检验来验证设计的有效性。

### 5.1 McCrary 密度检验（操纵检验）

**目的**：检验个体是否能够精确控制驱动变量，从而操纵自己的处理状态。

**逻辑**：
- 如果个体能操纵（如考试作弊刚好达到 60 分），驱动变量在门槛处的密度会有**不连续跳跃**
- 如果没有操纵，密度应该**平滑连续**

**方法**（McCrary, 2008）：
1. 将驱动变量分箱，计算每个箱的频数
2. 在门槛两侧分别拟合密度函数（局部线性）
3. 检验门槛处的密度跳跃是否显著

**检验统计量**：
$$
\theta = \log f_+(c) - \log f_-(c)
$$

其中 $f_+(c)$ 和 $f_-(c)$ 是门槛右侧和左侧的密度估计。

**零假设**: $H_0: \theta = 0$（密度连续）

### 5.2 协变量连续性检验

**目的**：检验个体特征在门槛处是否连续。

**逻辑**：
- 如果门槛附近的个体可比，他们的**基线特征**应该相似
- 对每个协变量 $X_k$，检验其在门槛处是否有跳跃

**方法**：
1. 对每个协变量，用 RDD 设计估计"伪效应"
2. 如果协变量连续，伪效应应接近 0

**示例协变量**：
- 用户年龄
- 注册时长
- 历史消费水平
- 地理位置

### 5.3 Placebo 检验

**目的**：检验在不应该有效应的地方是否也检测到了效应。

#### (a) 伪门槛检验
- 在真实门槛左右两侧选择伪门槛（如 c - 20, c + 20）
- 估计伪门槛处的"效应"
- 预期：伪效应应该不显著

#### (b) 伪结果检验
- 使用不应该受处理影响的结果变量（如用户性别、注册渠道）
- 估计处理对伪结果的"效应"
- 预期：伪效应应该不显著

### 5.4 检验汇总表

| 检验类型 | 检验内容 | 零假设 | 期望结果 |
|----------|----------|--------|----------|
| McCrary 密度 | 驱动变量密度连续 | 密度无跳跃 | 不拒绝 H0 |
| 协变量连续性 | 基线特征平滑 | 协变量无跳跃 | 不拒绝 H0 |
| 伪门槛 | 非门槛处无效应 | 伪效应 = 0 | 不拒绝 H0 |
| 伪结果 | 不相关结果无效应 | 伪效应 = 0 | 不拒绝 H0 |

---

In [ ]:
# 检验 1: McCrary 密度检验

def mccrary_test(X, cutoff, bins=30):
    """
    McCrary (2008) 密度检验
    
    返回:
        theta: log 密度差
        se: 标准误
        p_value: p 值
    """
    # 分箱
    bin_edges = np.linspace(X.min(), X.max(), bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    counts, _ = np.histogram(X, bins=bin_edges)
    
    # 密度估计（频数 / 总数 / 箱宽）
    bin_width = bin_edges[1] - bin_edges[0]
    density = counts / (len(X) * bin_width)
    
    # 找到门槛所在的箱
    cutoff_idx = np.searchsorted(bin_edges, cutoff) - 1
    
    # 左右两侧的密度（简化：取门槛左右的平均密度）
    left_density = density[:cutoff_idx].mean()
    right_density = density[cutoff_idx:].mean()
    
    # Log 密度差
    theta = np.log(right_density) - np.log(left_density)
    
    # 标准误（简化估计）
    se = np.sqrt(1/counts[:cutoff_idx].sum() + 1/counts[cutoff_idx:].sum())
    
    # p 值
    z_stat = theta / se
    p_value = 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    return theta, se, p_value, bin_centers, density

# 执行 McCrary 检验
theta, se, p_value, bin_centers, density = mccrary_test(df['spending'].values, cutoff=200)

# 可视化
fig = go.Figure()

fig.add_trace(go.Bar(
    x=bin_centers,
    y=density,
    marker_color=COLORS['primary'],
    opacity=0.7,
    name='密度'
))

fig.add_vline(x=200, line_dash="dash", line_color="black",
              annotation_text="门槛: 200 元")

fig.update_layout(
    title=f'McCrary 密度检验 (θ = {theta:.3f}, p = {p_value:.3f})',
    xaxis_title='消费金额 (元)',
    yaxis_title='密度',
    template='plotly_white',
    height=400
)

fig.show()

print("\n" + "="*60)
print("McCrary 密度检验")
print("="*60)
print(f"Log 密度差 (θ):     {theta:.4f}")
print(f"标准误 (SE):        {se:.4f}")
print(f"p 值:              {p_value:.4f}")
print("-"*60)
if p_value > 0.05:
    print("✅ 无法拒绝密度连续的零假设 → 无操纵证据")
else:
    print("⚠️  拒绝密度连续的零假设 → 可能存在操纵")
print("="*60)

In [ ]:
# 检验 2: 协变量连续性检验

# 生成带协变量的数据
np.random.seed(42)
n = len(df)
df['age'] = np.random.normal(35, 10, n)  # 年龄（不应该跳跃）
df['tenure_days'] = np.random.exponential(365, n)  # 注册天数（不应该跳跃）
df['avg_order_value'] = df['spending'] * 0.5 + np.random.normal(0, 20, n)  # 历史订单金额（不应该跳跃）

# 对每个协变量进行 RDD 估计
covariates = ['age', 'tenure_days', 'avg_order_value']
balance_results = []

for covar in covariates:
    rdd = SharpRDD(cutoff=200, bandwidth=50, polynomial_order=1)
    rdd.fit(df['spending'], df[covar])
    
    z_stat = rdd.tau_ / rdd.se_
    p_val = 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    balance_results.append({
        'Covariate': covar,
        'Estimate': rdd.tau_,
        'SE': rdd.se_,
        'p-value': p_val,
        'Balanced': '✅' if p_val > 0.05 else '❌'
    })

balance_df = pd.DataFrame(balance_results)

print("\n" + "="*70)
print("协变量连续性检验（平衡性检验）")
print("="*70)
print(balance_df.to_string(index=False))
print("="*70)
print("\n💡 解释: 如果协变量在门槛处有显著跳跃，说明门槛附近的个体不可比")

### 🎯 TODO 3: Placebo 检验

请完成伪门槛检验，验证在非真实门槛处不应该有效应。

In [ ]:
# ===================================# TODO 3: Placebo 检验 - 伪门槛 - 完整实现# ===================================# 真实门槛: 200# 伪门槛: 在真实门槛附近但不包括真实门槛的点placebo_cutoffs = [150, 170, 190, 210, 230, 250]placebo_results = []for cutoff_placebo in placebo_cutoffs:    # 对每个伪门槛进行 RDD 估计    rdd_placebo = SharpRDD(cutoff=cutoff_placebo, bandwidth=30, polynomial_order=1)    rdd_placebo.fit(df['score'].values, df['conversion'].values)        # 计算 p-value    t_stat = rdd_placebo.tau_ / rdd_placebo.se_    pvalue = 2 * (1 - stats.norm.cdf(abs(t_stat)))        placebo_results.append({        'cutoff': cutoff_placebo,        'effect': rdd_placebo.tau_,        'se': rdd_placebo.se_,        'pvalue': pvalue    })df_placebo = pd.DataFrame(placebo_results)# 真实门槛的结果rdd_real = SharpRDD(cutoff=200, bandwidth=30, polynomial_order=1)rdd_real.fit(df['score'].values, df['conversion'].values)t_stat_real = rdd_real.tau_ / rdd_real.se_pvalue_real = 2 * (1 - stats.norm.cdf(abs(t_stat_real)))# 打印结果print("=" * 80)print("Placebo Test 结果（伪门槛检验）")print("=" * 80)print(f"{'伪门槛':>10s} {'效应':>10s} {'标准误':>10s} {'P-value':>10s} {'显著性':>10s}")print("-" * 80)for _, row in df_placebo.iterrows():    sig = '***' if row['pvalue'] < 0.01 else ('**' if row['pvalue'] < 0.05 else ('*' if row['pvalue'] < 0.1 else ''))    print(f"{row['cutoff']:10.0f} {row['effect']:10.4f} {row['se']:10.4f} {row['pvalue']:10.4f} {sig:>10s}")# 真实门槛print("-" * 80)sig_real = '***' if pvalue_real < 0.01 else ('**' if pvalue_real < 0.05 else ('*' if pvalue_real < 0.1 else ''))print(f"{'200 (真实)':>10s} {rdd_real.tau_:10.4f} {rdd_real.se_:10.4f} {pvalue_real:10.4f} {sig_real:>10s}")print("=" * 80)# 可视化fig = make_subplots(    rows=1, cols=2,    subplot_titles=('效应估计值', 'P-value 分布'))# 左图：效应估计# 伪门槛fig.add_trace(    go.Scatter(        x=df_placebo['cutoff'],        y=df_placebo['effect'],        error_y=dict(            type='data',            array=1.96*df_placebo['se'],            visible=True        ),        mode='markers',        name='伪门槛',        marker=dict(size=10, color=COLORS['synthetic']),    ),    row=1, col=1)# 真实门槛fig.add_trace(    go.Scatter(        x=[200],        y=[rdd_real.tau_],        error_y=dict(            type='data',            array=[1.96*rdd_real.se_],            visible=True        ),        mode='markers',        name='真实门槛',        marker=dict(size=15, color=COLORS['treated'], symbol='diamond'),    ),    row=1, col=1)# 零线fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)# 真实效应水平线fig.add_hline(y=rdd_real.tau_, line_dash="dot", line_color=COLORS['treated'],               opacity=0.5, row=1, col=1)# 右图：P-valuesfig.add_trace(    go.Scatter(        x=df_placebo['cutoff'],        y=df_placebo['pvalue'],        mode='lines+markers',        name='伪门槛',        line=dict(color=COLORS['synthetic'], width=2),        marker=dict(size=10),        showlegend=False    ),    row=1, col=2)# 真实门槛的 p-valuefig.add_trace(    go.Scatter(        x=[200],        y=[pvalue_real],        mode='markers',        name='真实门槛',        marker=dict(size=15, color=COLORS['treated'], symbol='diamond'),        showlegend=False    ),    row=1, col=2)# 显著性水平线fig.add_hline(y=0.05, line_dash="dash", line_color="red",               annotation_text="α=0.05", row=1, col=2)fig.update_xaxes(title_text="门槛值", row=1, col=1)fig.update_xaxes(title_text="门槛值", row=1, col=2)fig.update_yaxes(title_text="RDD 估计量", row=1, col=1)fig.update_yaxes(title_text="P-value", row=1, col=2)fig.update_layout(    title_text='Placebo Test: 伪门槛检验',    template='plotly_white',    height=400,    showlegend=True)fig.show()# 统计检验n_sig_placebo = (df_placebo['pvalue'] < 0.05).sum()n_total_placebo = len(df_placebo)print(f"\n💡 解读:")print(f"  • 在 {n_total_placebo} 个伪门槛中，{n_sig_placebo} 个在 5% 水平上显著")print(f"  • 理想情况：伪门槛处效应不显著，只有真实门槛处显著")if n_sig_placebo == 0:    print(f"  ✓ 所有伪门槛均不显著，Placebo Test 通过")    print(f"  ✓ 这增强了我们对真实效应的信心")elif n_sig_placebo <= n_total_placebo * 0.05:    print(f"  ✓ 少数伪门槛显著（≤5%），在预期范围内")    print(f"  ✓ Placebo Test 基本通过")else:    print(f"  ⚠ 较多伪门槛显著（>{n_total_placebo * 0.05:.0f}个），需要警惕")    print(f"  ⚠ 可能原因:")    print(f"    1. 函数形式设定错误（应该用更高阶多项式）")    print(f"    2. 存在其他未观测的跳跃点")    print(f"    3. 数据中存在其他结构性变化")print(f"\n  • 真实门槛处 p-value = {pvalue_real:.4f}")if pvalue_real < 0.05:    print(f"  ✓ 真实门槛处效应显著（p < 0.05）")else:    print(f"  ✗ 真实门槛处效应不显著（p >= 0.05）")

---

## Part 6: 业务案例

### 案例 1: 优惠券门槛效应（满减活动）

**背景**：电商平台推出「满 200 减 50」活动，希望评估优惠券对用户复购的长期效应。

**数据**：
- 驱动变量：首单消费金额
- 处理：是否获得 50 元优惠券
- 结果：30 天内复购率

**挑战**：
- 用户可能为了凑单而刚好超过 200 元（操纵）
- 需要检验密度连续性

---

In [ ]:
# 案例 1: 优惠券门槛效应

# 模拟真实业务数据（考虑轻微操纵）
def simulate_coupon_case(n=2000):
    np.random.seed(123)
    
    # 自然消费（无操纵）
    natural_spending = np.random.gamma(shape=4, scale=50, size=int(n*0.9))
    
    # 凑单消费（轻微操纵，集中在 200-210 之间）
    manipulated_spending = np.random.uniform(200, 210, int(n*0.1))
    
    spending = np.concatenate([natural_spending, manipulated_spending])
    spending = spending[(spending >= 100) & (spending <= 300)]  # 过滤极端值
    
    # 处理
    coupon = (spending >= 200).astype(int)
    
    # 协变量
    age = np.random.normal(32, 8, len(spending))
    is_member = np.random.binomial(1, 0.3, len(spending))
    
    # 结果（复购率受年龄、会员、优惠券影响）
    base_repurchase = 25 + 0.15 * (spending - 200) - 0.3 * age + 10 * is_member
    treatment_effect = 12  # 真实效应 12%
    
    repurchase_rate = base_repurchase + treatment_effect * coupon + np.random.normal(0, 15, len(spending))
    repurchase_rate = np.clip(repurchase_rate, 0, 100)
    
    return pd.DataFrame({
        'spending': spending,
        'coupon': coupon,
        'repurchase_rate': repurchase_rate,
        'age': age,
        'is_member': is_member
    })

df_case1 = simulate_coupon_case(n=2000)

print("\n" + "="*60)
print("案例 1: 优惠券门槛效应分析")
print("="*60)

# 步骤 1: McCrary 密度检验
print("\n[步骤 1] McCrary 密度检验")
theta, se, p_value, _, _ = mccrary_test(df_case1['spending'].values, cutoff=200)
print(f"  Log 密度差: {theta:.4f} (p = {p_value:.4f})")
if p_value < 0.05:
    print("  ⚠️  检测到密度跳跃，可能存在凑单行为")
else:
    print("  ✅ 密度连续，无明显操纵")

# 步骤 2: 协变量平衡
print("\n[步骤 2] 协变量平衡性检验")
for covar in ['age', 'is_member']:
    rdd = SharpRDD(cutoff=200, bandwidth=40)
    rdd.fit(df_case1['spending'], df_case1[covar])
    p = 2 * (1 - stats.norm.cdf(np.abs(rdd.tau_ / rdd.se_)))
    status = "✅" if p > 0.05 else "❌"
    print(f"  {covar}: τ = {rdd.tau_:.4f} (p = {p:.4f}) {status}")

# 步骤 3: RDD 估计
print("\n[步骤 3] RDD 效应估计")
rdd_main = SharpRDD(cutoff=200, bandwidth=None)  # 自动选择带宽
rdd_main.fit(df_case1['spending'], df_case1['repurchase_rate'])
results_case1 = rdd_main.summary()

print(f"\n📊 结论: 优惠券使复购率提升 {results_case1['tau']:.2f}% ")
print(f"         (95% CI: [{results_case1['ci'][0]:.2f}, {results_case1['ci'][1]:.2f}])")
print(f"\n💡 业务建议: 满减活动有效，可考虑长期化或调整门槛")

### 案例 2: 会员等级效应

**背景**：平台设置会员等级，消费满 5000 元升级为金卡会员，享受专属权益。评估金卡权益对用户活跃度的影响。

**数据**：
- 驱动变量：年度累计消费
- 处理：是否为金卡会员
- 结果：次月活跃天数

**特点**：
- Fuzzy RDD（有人达标但未升级，有人未达标但客服特批）

---

In [ ]:
# 案例 2: 会员等级效应（Fuzzy RDD）

def simulate_membership_case(n=1500):
    np.random.seed(456)
    
    # 年度消费
    annual_spending = np.random.gamma(shape=3, scale=1500, size=n)
    annual_spending = annual_spending[(annual_spending >= 2000) & (annual_spending <= 8000)]
    
    # 有资格升级（超过 5000）
    eligible = (annual_spending >= 5000).astype(int)
    
    # 实际升级（Fuzzy）
    # 达标: 80% 升级
    # 未达标: 15% 特批升级
    actual_upgrade = np.zeros(len(annual_spending))
    actual_upgrade[eligible == 1] = np.random.binomial(1, 0.8, (eligible == 1).sum())
    actual_upgrade[eligible == 0] = np.random.binomial(1, 0.15, (eligible == 0).sum())
    
    # 结果（活跃天数）
    base_activity = 10 + 0.002 * (annual_spending - 5000) + np.random.normal(0, 3, len(annual_spending))
    treatment_effect = 5  # 金卡使活跃天数增加 5 天
    
    activity_days = base_activity + treatment_effect * actual_upgrade
    activity_days = np.clip(activity_days, 0, 30)
    
    return pd.DataFrame({
        'annual_spending': annual_spending,
        'eligible': eligible,
        'gold_member': actual_upgrade,
        'activity_days': activity_days
    })

df_case2 = simulate_membership_case(n=1500)

print("\n" + "="*60)
print("案例 2: 会员等级效应分析 (Fuzzy RDD)")
print("="*60)

# 检查处理概率跳跃
bandwidth = 1500
df_bw = df_case2[np.abs(df_case2['annual_spending'] - 5000) <= bandwidth].copy()

left_treatment_prob = df_bw[df_bw['annual_spending'] < 5000]['gold_member'].mean()
right_treatment_prob = df_bw[df_bw['annual_spending'] >= 5000]['gold_member'].mean()
first_stage_jump = right_treatment_prob - left_treatment_prob

print(f"\n[第一阶段] 处理概率跳跃")
print(f"  左侧升级率: {left_treatment_prob:.3f}")
print(f"  右侧升级率: {right_treatment_prob:.3f}")
print(f"  跳跃幅度:   {first_stage_jump:.3f}")

# 2SLS 估计
df_bw['X_centered'] = df_bw['annual_spending'] - 5000
df_bw['Z_X'] = df_bw['eligible'] * df_bw['X_centered']

# 第一阶段
X_first = df_bw[['eligible', 'X_centered', 'Z_X']].values
D_bw = df_bw['gold_member'].values
first_stage = LinearRegression().fit(X_first, D_bw)
D_hat = first_stage.predict(X_first)

# 第二阶段
X_second = np.column_stack([D_hat, df_bw[['X_centered', 'Z_X']].values])
Y_bw = df_bw['activity_days'].values
second_stage = LinearRegression().fit(X_second, Y_bw)
tau_2sls = second_stage.coef_[0]

# 标准误（简化）
residuals = Y_bw - second_stage.predict(X_second)
se_2sls = np.sqrt(np.sum(residuals**2) / (len(Y_bw) - 4)) / np.sqrt(len(Y_bw))

print(f"\n[第二阶段] 2SLS 估计")
print(f"  LATE (顺从者效应): {tau_2sls:.3f} 天")
print(f"  标准误:            {se_2sls:.3f}")
print(f"  95% CI:           [{tau_2sls - 1.96*se_2sls:.3f}, {tau_2sls + 1.96*se_2sls:.3f}]")

print(f"\n📊 结论: 金卡会员使活跃天数增加 {tau_2sls:.1f} 天（针对顺从者）")
print(f"\n💡 业务解释: 这是对'因金卡权益而升级的用户'的效应，不包括本来就很活跃的用户")

### 案例 3: 信用评分门槛

**背景**：金融平台设置信用评分 600 分为门槛，600 分以上可享受更低利率。评估利率优惠对借款金额的影响。

**数据**：
- 驱动变量：信用评分（590-610 区间）
- 处理：是否享受低利率
- 结果：借款金额

**要点**：
- 窄带宽（信用评分精确度高）
- 需要检验评分操纵（如虚假信息）

---

In [ ]:
# 案例 3: 信用评分门槛

def simulate_credit_case(n=1000):
    np.random.seed(789)
    
    # 信用评分（集中在 590-610 附近）
    credit_score = np.random.normal(600, 5, n)
    credit_score = credit_score[(credit_score >= 590) & (credit_score <= 610)]
    
    # 处理（低利率）
    low_rate = (credit_score >= 600).astype(int)
    
    # 结果（借款金额，万元）
    base_loan = 20 + 0.5 * (credit_score - 600) + np.random.normal(0, 5, len(credit_score))
    treatment_effect = 8  # 低利率使借款金额增加 8 万
    
    loan_amount = base_loan + treatment_effect * low_rate
    loan_amount = np.clip(loan_amount, 5, 50)
    
    return pd.DataFrame({
        'credit_score': credit_score,
        'low_rate': low_rate,
        'loan_amount': loan_amount
    })

df_case3 = simulate_credit_case(n=1000)

print("\n" + "="*60)
print("案例 3: 信用评分门槛效应分析")
print("="*60)

# RDD 估计（窄带宽）
rdd_case3 = SharpRDD(cutoff=600, bandwidth=5, polynomial_order=1)
rdd_case3.fit(df_case3['credit_score'], df_case3['loan_amount'])
results_case3 = rdd_case3.summary()

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_case3[df_case3['low_rate']==0]['credit_score'],
    y=df_case3[df_case3['low_rate']==0]['loan_amount'],
    mode='markers',
    name='普通利率',
    marker=dict(color=COLORS['danger'], size=5, opacity=0.6)
))

fig.add_trace(go.Scatter(
    x=df_case3[df_case3['low_rate']==1]['credit_score'],
    y=df_case3[df_case3['low_rate']==1]['loan_amount'],
    mode='markers',
    name='优惠利率',
    marker=dict(color=COLORS['secondary'], size=5, opacity=0.6)
))

fig.add_vline(x=600, line_dash="dash", line_color="black",
              annotation_text="门槛: 600 分")

fig.update_layout(
    title=f'信用评分门槛效应: {results_case3["tau"]:.2f} 万元',
    xaxis_title='信用评分',
    yaxis_title='借款金额 (万元)',
    template='plotly_white',
    height=400
)

fig.show()

print(f"\n📊 结论: 低利率使借款金额增加 {results_case3['tau']:.2f} 万元")
print(f"         (95% CI: [{results_case3['ci'][0]:.2f}, {results_case3['ci'][1]:.2f}])")
print(f"\n💡 业务建议: 利率优惠有效刺激借款需求，可考虑动态调整门槛")

---

## 练习与思考题

### 练习 1: RDD 估计器对比

使用 `df` 数据，对比以下估计器的结果：
1. 线性规范（`polynomial_order=1`）
2. 二次规范（`polynomial_order=2`）
3. 不同带宽（20, 50, 100）

**问题**：哪种规范最稳健？为什么？

---

### 练习 2: Fuzzy RDD 的 LATE 解释

在案例 2（会员等级）中：
1. 计算顺从者 (Compliers) 的比例
2. 与 Always-takers 和 Never-takers 的比例对比
3. 讨论：LATE 能否推广到总体？

---

### 练习 3: 带宽选择的影响

绘制带宽从 10 到 100 的敏感性图，观察：
1. 点估计如何变化
2. 置信区间如何变化
3. 在什么带宽下结果最稳定

---

### 思考题 1: RDD 的局限性

**问题**：RDD 只能识别门槛处的局部效应 (LATE)，无法推断到远离门槛的人群。在业务决策中，如何应对这一局限？

**提示**：考虑外部效度 vs 内部效度的权衡。

---

### 思考题 2: 操纵检验的重要性

**场景**：某教育平台设置"考试 60 分及格"的门槛。你发现在 60 分处有明显的密度跳跃（很多学生刚好 60 分）。

**问题**：
1. 这说明了什么？
2. RDD 设计还有效吗？
3. 如何补救或调整分析策略？

---

### 思考题 3: RDD 在 A/B 测试中的应用

**场景**：你的 A/B 测试因技术故障，只有部分用户被随机分配，其余用户根据"注册时间超过 30 天"分配到对照组。

**问题**：
1. 能否用 RDD 补救这个实验？
2. 需要满足什么假设？
3. 如何与完全随机的子样本结合分析？

---

## 总结

### RDD 的核心要点

1. **核心思想**：利用政策/规则产生的门槛，在断点处获得局部随机化
2. **关键假设**：结果变量在门槛处连续（无处理时）
3. **Sharp vs Fuzzy**：完全跳跃 vs 概率跳跃，后者等价于 IV
4. **带宽选择**：平衡偏差和方差，使用 IK 或 CCT 方法
5. **有效性检验**：密度检验、协变量平衡、Placebo 检验

### 实践建议

1. **可视化第一**：画出数据再估计
2. **多重稳健性**：报告不同带宽、不同规范的结果
3. **检验假设**：不要跳过有效性检验
4. **谨慎外推**：RDD 只识别门槛处的局部效应
5. **业务结合**：理解门槛的业务含义，判断可信度

### 延伸阅读

- **Imbens & Lemieux (2008)**: "Regression Discontinuity Designs: A Guide to Practice"
- **Lee & Lemieux (2010)**: "Regression Discontinuity Designs in Economics"
- **Calonico, Cattaneo & Titiunik (2014)**: "Robust Nonparametric Confidence Intervals for RDD"
- **Cattaneo, Idrobo & Titiunik (2019)**: *A Practical Introduction to RDD* (书籍)

---

**恭喜你完成了 RDD 的学习！** 🎉

你现在掌握了利用"门槛"识别因果效应的强大工具。下一步，我们将学习**合成控制法 (Synthetic Control)**，应对没有明确门槛的场景。